In [6]:
import numpy as np
import mne
import mne_connectivity # import spectral_connectivity
import scipy.io
import os
from glob import glob

In [7]:
# Get stc only from selected labels
roi_names = [# Left
             'rostralanteriorcingulate-lh', # Left Rostral ACC
             'caudalanteriorcingulate-lh', # Left Caudal ACC
             'postcentral-lh', # Left S1,
             'insula-lh', 'superiorfrontal-lh', # Left Insula, Left DL-PFC,
             'medialorbitofrontal-lh', # Left Medial-OFC
             # Right
             'rostralanteriorcingulate-rh', # Right Rostral ACC
             'caudalanteriorcingulate-rh', # Right Caudal ACC
             'postcentral-rh', # , Right S1
             'insula-rh', 'superiorfrontal-rh', # Right Insula, Right DL-PFC
             'medialorbitofrontal-rh'] # Right Medial-OFC

In [11]:
# Load the MATLAB file
sub_id='015'
data_path = f'/../../Data/STC_Z_scored_Epochs/5_sec_time_window/{sub_id}/'

file_path = glob(data_path, '*rh_stc_zepo.mat')
# 'caudalanteriorcingulate-rh_stc_zepo.mat'
data = scipy.io.loadmat(file_path)

# Display the keys and structure of the data
data.keys(), {key: type(data[key]) for key in data.keys()}

# Examine the shape and basic information of the 'data' array
data_shape = data['data'].shape
data_info = data['data']

# Display the shape and some basic info about the data
data_shape, data_info.dtype, data_info.ndim

TypeError: glob() takes 1 positional argument but 2 were given

In [ ]:
# Reshape the data to be compatible with MNE-Connectivity
# Assuming each sample is a separate time point and each trial is independent

# Current shape is (trials, samples), need (trials, channels, samples)
# Since we have only one channel (region), we add an extra dimension
data_reshaped = data['data'][:, np.newaxis, :]

# Create artificial data for a second region (placeholder)
# In real analysis, this should be different data from another region
data_region2 = np.copy(data_reshaped)

# Verify the new shape
data_reshaped.shape, data_region2.shape


In [ ]:
# Assuming data_reshaped is your EEG data reshaped to (trials, channels, samples)
# data_region2 is the placeholder for the second region, identical to data_reshaped

# Concatenate the two regions along the channel dimension
combined_data = np.concatenate([data_reshaped, data_region2], axis=1)

# Create an MNE EpochsArray for compatibility with MNE-Connectivity
info = mne.create_info(['region1', 'region2'], sfreq=100, ch_types='eeg')  # sfreq is the sampling frequency
epochs = mne.EpochsArray(combined_data, info)

# Compute wPLI
fmin, fmax = 8, 13  # Define frequency band (e.g., alpha band)
sfreq = epochs.info['sfreq']  # Sampling frequency
con, freqs, times, n_epochs, n_tapers = spectral_connectivity(
    epochs, method='wpli2_debiased', mode='multitaper', sfreq=sfreq,
    fmin=fmin, fmax=fmax, faverage=True, mt_adaptive=True, n_jobs=1)

# con contains the wPLI values between the two regions
print(con)
